In [1]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore
!pip install Flask-Mail
!pip install scikit-surprise

  Using cached https://files.pythonhosted.org/packages/b8/6c/bedcab39e8dc969f7a48d058dbacd69fc07ce3f817a03de875902016f667/google_cloud_datastore-1.15.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b9/be/66b4b7c7e94879257f404a716385c62c0629e011351009c646dd04813175/google_cloud_core-1.4.4-py2.py3-none-any.whl
google-cloud-core 1.4.4 has requirement six>=1.12.0, but you'll have six 1.10.0 which is incompatible.
google-cloud-bigquery 1.9.0 has requirement google-cloud-core<0.30dev,>=0.29.0, but you'll have google-cloud-core 1.4.4 which is incompatible.
  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstalled google-cloud-core-0.28.1
  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl


In [1]:

import json
import re
from flask import Flask, request, jsonify, make_response
from flask import render_template
from google.cloud import datastore

import os
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import BaselineOnly
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import pandas as pd

In [ ]:
from send_invitation import send_email 
from threading import Thread
import jwt

app = Flask(__name__)
results = ""
rsize = 0
current_user_id = 0
#Quantity of the recommond results
top_k = 5
recResults = []

import re
from random import seed
from random import randint
from datetime import datetime

seed(datetime.now())

app.config['SECRET_KEY'] = 'SECRET_KEY_CHAT9900'

file_path = os.path.expanduser('ratings_small.csv')
movie_path = "movies_metadata.csv"
# friend_path = "friends.csv"

reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)
surprise_data = Dataset.load_from_file(file_path, reader=reader)
movies_df = pd.read_csv(movie_path)

#chatroom public url
def get_chatroom_url():
    # populate this from your local server
    return "http://d5a7fa138cf2.ngrok.io"

# url of the webhook. 
# it is done by curl the local ngrok api
# It is for friend request validations [out of scope]
def get_public_url():
    tunnel = !curl https://4040-cs-269605604529-default.cs-asia-southeast1-vjqr.cloudshell.dev/api/tunnels
    tunnel= str(tunnel)
    print(tunnel)
    mobj = re.search('public_url', tunnel)
    i = mobj.span()
    t = tunnel[i[1]:].split('\"')
    public_url =t[2]
    return public_url

def gen_jwt(seedstr):
    return jwt.encode(
            {'requester': seedstr },
            app.config['SECRET_KEY'], algorithm='HS256')

# [out of scope]
@app.route('/friend/<token>', methods=['GET', 'POST'])
def validate_friend_requests(token):
    meeting = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['requester']
    print("meeting click was clicked ",meeting)
    return render_template('place_holder.html')

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    intent = req.get('queryResult').get('intent').get('displayName')
    print(intent)
    # there could be multiple topics in a sentence
    if isinstance(intent, list):
        intent = intent[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    
    rsp = buildReply('Command not recognized')
    if (intent == "Schedule: Movie - yes"):
        rsp = scheduleMovie(req)
    elif (intent == "profile" or intent == "account"):
        rsp = getProfileAccount()
    elif (intent == "Recommendation:" ):
        #
        rsp = getRecommendations()
    elif (intent == "login-prompt"):
        rsp = validateLogin(req)
        
    rsp = json.dumps(rsp, indent=4)
    print (rsp)
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
# get similar users
def getSimilarUsers(top_k,u_id):
    all_trainset = surprise_data.build_full_trainset()
    algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': True}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    algo.fit(all_trainset)
    user_inner_id = algo.trainset.to_inner_uid(u_id)
    user_neighbors = algo.get_neighbors(user_inner_id, k=top_k)
    user_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in user_neighbors)
    return user_neighbors

# get similar movies
def getSimilarItems(top_k,item_id):
    all_trainset = surprise_data.build_full_trainset()
    item_algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': False}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    item_algo.fit(all_trainset)
    item_inner_id = item_algo.trainset.to_inner_iid(item_id)
    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)
    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)
                       for inner_id in item_neighbors)
    return f_item_neighbors

def getMoviesFromList(movieList):
    movies = []
    text = ""
    for movie in movieList:
        #print(movies)
        for i in range(len(movies_df)):
            if movies_df['id'][i] == movie:
                movies.append(movies_df['title'][i])
                if movie not in movies_df['id']:
                    movies.append("Movie ID"+movies_df['id'][i]+"(currently unavailable)")
                
    for m in movies:
        text += str(m)+"\n"
    print(text)
    return text

def getMoviesFromTitle(title):
    #get movie ID or report error
    movies = []
    for i in range(len(movies_df)):
        if str(movies_df['title'][i]) == str(title):
            moviesList = list(getSimilarItems(top_k,movies_df['id'][i]))
            break
        elif i==len(movies_df)-1:
            return print("Couldn't find the movie, Please try again.")
    print(moviesList)
    return getMoviesFromList(moviesList)
    
def getMoviesFromUser(userID):
    user_df = pd.read_csv('ratings_small.csv')
    maxRate = 0
    for i in range(len(user_df)):
        if (str(user_df['userId'][i]) == str(userID)) and (float(user_df['rating'][i]) > maxRate):
            maxRate = user_df['rating'][i]
            likedID = user_df['movieId'][i]

    recList = list(getSimilarItems(top_k,str(likedID)))
    return getMoviesFromList(recList)


def get_attendees(current_user_id, current_user_email, tofriend):
    attendees = list()
    attendees.append("chatbot9900@gmail.com")
    attendees.append(current_user_email)
    client = datastore.Client()
    query = client.query(kind="friends")
    query.add_filter("id", "=", current_user_id)
    result = list(query.fetch(1))
    print(current_user_id, result)
    print('fids',result[0].get('fids'))
    friends = result[0].get('fids')
    for f in friends:
        query = client.query(kind="users")
        query.add_filter("id", "=", int(f))
        query.add_filter("name", "=", tofriend)
        print(query)
        print(f, tofriend)
        result = list(query.fetch(1))
        if len(result) == 1 :
            print('email',result[0].get('email'))
            attendees.append(result[0].get('email'))
    return attendees
        
def scheduleMovie(req):
    tofriend = req.get('queryResult').get('outputContexts')[0].get('parameters').get('friend')[0].get('name')
    print(tofriend,'tofriend')
    movie = req.get('queryResult').get('outputContexts')[0].get('parameters').get('movie')[0]
    date = req.get('queryResult').get('outputContexts')[0].get('parameters').get('date')[0]
    time = req.get('queryResult').get('outputContexts')[0].get('parameters').get('time')[0]
    current_user_email = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user')
    #
    print(current_user_email, tofriend)
    if (current_user_email == None):
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    
    client = datastore.Client()
    query = client.query(kind="users")
    query.add_filter("email", "=", current_user_email)
    result = list(query.fetch(1))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "logged in user email not found. Please contact admin chatbotchatbot9900@gmail.com"
        }
    #print(result)
    #print(result[0])
    #print(result[0].id)
    #print(result[0].get('email'))
    #print(attendees)
    attendees = get_attendees(result[0].get("id"), current_user_email, tofriend)
    #print(str(datetime.now()))
    print(attendees)
    
    # token in this context is just a random string acting as chatroom id
    token = gen_jwt(str(datetime.now()))
    chatroom_url = get_chatroom_url()+"/"+token.decode("utf-8")
    Thread(target=send_email, args=(attendees,chatroom_url, movie, date.split('T')[0], time.split('T')[1]),).start()

    return buildReply('Awesome! Your movie meeting is confirmed. Sending request...')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print (urls)
    return buildReply(urls)

'''
def getRecommendations():
    global results 
    global rsize
    print ("recom")
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print(rsize)
    t1 = results[randint(0, rsize -1 )]['title'].decode("utf-8") 
    print("t1",t1)
    t2 = results[randint(0, rsize-1)]['title'].decode("utf-8") 
    return buildReply(t1+"\n"+t2)
'''

def getRecommendations():
    global recResults
    global current_user_id
    if current_user_id==None:
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    return buildReply(str(recResults))
    
def getRecResults(uid):
    global recResults
    global current_user_id
    #get user id
    current_user_id = uid
    recResults = getMoviesFromUser(current_user_id)
    #get rec list stored
    return recResults
    
def validateLogin(req):
    email = req.get('queryResult').get('parameters').get("email")
    password = req.get('queryResult').get('parameters').get("password")
    session = req.get('queryResult').get('outputContexts')[0].get('name')
    client = datastore.Client()
    # you may have to create this kind manually in gcp datastore 
    query = client.query(kind="users")
    print(email,password)
    # These two columns has to be indexed in gcp datastore 
    # otherwise they can not be found and filtered
    query.add_filter("email", "=", email)
    query.add_filter("randomnumber", "=", password)
    result = list(query.fetch(1))
        
    #New thread for recommendation
    #Thread(target=getRecResults,args=(req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user_id'))).start()
   
    
    print("validateLogin",result)
    if len(result) == 1:
        id = result[0].get("id")
        uid = id
        #New thread for recommendation
        Thread(target=getRecResults,args=(uid)).start()
        return confirmLogin(id, email, session)
    return buildReply("Invalid combination")
  
def confirmLogin(id, un, sn):
    array = sn.split('/')
    session = ""
    for i in range(len(array)-1):
        session += array[i]+'/'
    session += "logged-in"
    return {
      "fulfillmentText": 
              "User: "+un+" logged in"
      ,
      "outputContexts": [
        {
          "name": session,
          "lifespanCount": 500,
          "parameters": {
            "loggedin_user": un,
            "loggedin_user_id": id
          }
        }
      ]
    }

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

/usr/local/envs/py3env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Request:
{
    "originalDetectIntentRequest": {
        "payload": {}
    },
    "queryResult": {
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        ""
                    ]
                }
            }
        ],
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.3,
                "score": 0.3
            }
        },
        "allRequiredParamsPresent": true,
        "languageCode": "en",
        "intentDetectionConfidence": 0.3,
        "queryText": "3274",
        "outputContexts": [
            {
                "parameters": {
                    "email.original": "Fannie.Rothfuss@zoozoo.nl",
                    "no-input": 0.0,
                    "email": "Fannie.Rothfuss@zoozoo.nl",
                    "password.original": "3274",
                    "no-match": 0.0,
                    "password": "3274"
                },
                "name":

127.0.0.1 - - [15/Apr/2021 14:53:20] "POST /webhook/ HTTP/1.1" 200 -


validateLogin [<Entity('users', 5632499082330112) {'name': 'Fannie', 'id': '1', 'randomnumber': '3274', 'email': 'Fannie.Rothfuss@zoozoo.nl'}>]
{
    "fulfillmentText": "User: Fannie.Rothfuss@zoozoo.nl logged in",
    "outputContexts": [
        {
            "lifespanCount": 500,
            "parameters": {
                "loggedin_user": "Fannie.Rothfuss@zoozoo.nl",
                "loggedin_user_id": "1"
            },
            "name": "projects/i-backbone-307006/agent/sessions/dfMessenger-11076048/contexts/logged-in"
        }
    ]
}
Computing the msd similarity matrix...
Done computing similarity matrix.
Training Day
Movie ID2034(currently unavailable)
Savior
Movie ID2042(currently unavailable)
La Chienne
Movie ID26614(currently unavailable)



127.0.0.1 - - [15/Apr/2021 14:53:40] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "originalDetectIntentRequest": {
        "payload": {}
    },
    "queryResult": {
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.2,
                "score": 0.2
            }
        },
        "fulfillmentMessages": [
            {
                "text": {
                    "text": [
                        "Sure! I can help you with that. Here are some of the top movies/TV shows that are currently trending at your location. Pick something that you would like to watch:"
                    ]
                }
            },
            {
                "payload": {
                    "richContent": [
                        [
                            {
                                "type": "chips",
                                "options": [
                                    {
                                        "text": "Back to Main menu"
                                    }
                   